In [159]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bertopic import BERTopic

from nltk import word_tokenize
import os.path as op
import re 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
import nltk

In [160]:
# import the preprocess doc

df = pd.read_csv('./Data_csv/data_preprocessed.csv')

In [161]:
df.columns

Index(['doc_id', 'Name of the document', 'Institution', 'URL', 'Authors',
       'Affiliates', 'Sector', 'Country', 'Date', 'Keywords',
       'Exclusion criteria', 'Status', 'Label', 'MapAIE (ours)', 'Jobin',
       'Fjeld', 'Tidjon', 'Hagendorff', 'Floridi', 'Zeng (LAIP)',
       'Attard-Frost', 'EP', 'Algorithm watch', 'CE', 'Winfield',
       'EthicalML GitHub', 'all sources', 'Checked by', 'Unnamed: 27',
       'Unnamed: 28', 'Unnamed: 29', 'text', 'langue', 'text_processed',
       'tfidf', 'categorie Institution', 'theme'],
      dtype='object')

In [162]:
df.count()

doc_id                   457
Name of the document     457
Institution              457
URL                      457
Authors                   33
Affiliates                29
Sector                    42
Country                  446
Date                      44
Keywords                   6
Exclusion criteria         0
Status                   457
Label                    394
MapAIE (ours)            457
Jobin                     59
Fjeld                     21
Tidjon                    24
Hagendorff                12
Floridi                    4
Zeng (LAIP)               42
Attard-Frost              28
EP                         8
Algorithm watch          103
CE                       377
Winfield                  15
EthicalML GitHub          13
all sources              457
Checked by               457
Unnamed: 27                2
Unnamed: 28                0
Unnamed: 29                2
text                     457
langue                   457
text_processed           457
tfidf         

In [163]:
documents = df['text_processed'].to_list()

In [ ]:
vectorizer = CountVectorizer(stop_words='english')
Bow = vectorizer.fit_transform(documents)

# Calculer la fréquence totale de chaque mot
frequency = Bow.toarray().sum(axis=0)

# Obtenir les 20 mots les plus fréquents
top_indices = np.argsort(frequency)[-20:]  # Indices des 20 mots les plus fréquents
vocabulary = np.array(vectorizer.get_feature_names_out())  # Le vocabulaire
top_words = vocabulary[top_indices] 

stopwords_to_remove = set(top_words)

dates = set([str(i) for i in range(1990, 2021)])
useless_words = set(('may','also','see'))
html_stopwords = set({'maxwidth', 'px', 'maxheight', 'img', 'src', 'https', 'com', 'www', 'http', 'jpg', 'png', 'gif', 'jpeg', 'pdf', 'html'})
css_stopwords = set({'grid','body', ':root', '\n', 'padding',',button','margintop', 'px', 'margin', 'border', 'width', 'height', 'color', 'font', 'size', 'background', 'position', 'left', 'right', 'top', 'bottom', 'display', 'flex', 'align', 'justify', 'content', 'center', 'float', 'clear', 'overflow', 'hidden', 'zindex', 'cursor', 'pointer', 'hover', 'active', 'focus', 'transition', 'transform', 'rotate', 'scale', 'translate', 'opacity', 'box', 'shadow', 'outline', 'none', 'block', 'inline', 'inlineblock', 'relative', 'absolute', 'fixed', 'static', 'sticky', 'visible', 'hidden'})

stopwords_to_remove = stopwords_to_remove.union(dates)
stopwords_to_remove = stopwords_to_remove.union(html_stopwords)
stopwords_to_remove = stopwords_to_remove.union(css_stopwords)
stopwords_to_remove = stopwords_to_remove.union(useless_words)




def remove_frequent_words(doc, stopwords):
    words = doc.split()
    cleaned_words = [word for word in words if word not in stopwords]
    return " ".join(cleaned_words)


documents = [remove_frequent_words(doc, stopwords_to_remove) for doc in documents]



In [165]:
import hdbscan

hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=20, min_samples=1, cluster_selection_epsilon=0.01)
topic_model = BERTopic(hdbscan_model=hdbscan_model, language="english", nr_topics=10)

In [166]:
n_topics = 0
while n_topics < 10:
    topics, probs = topic_model.fit_transform(documents)
    n_topics = len(set(topics))

In [167]:
df['topic'] = topics
df['probs'] = probs

In [168]:
# Afficher les topics les plus fréquents
print(topic_model.get_topic_info())



   Topic  Count                                           Name  \
0     -1     99         -1_also_government_technologies_report   
1      0     34                   0_system_also_may_protection   
2      1     23            1_uk_government_strategy_innovation   
3      2     97                2_ethics_ethical_machine_system   
4      3     20             3_biometric_facial_law_recognition   
5      4     35  4_cooperation_international_government_global   
6      5     31   5_algorithmic_algorithms_bias_decisionmaking   
7      6     35             6_model_protection_project_privacy   
8      7     41                     7_law_legal_europe_council   
9      8     42                  8_energy_policy_national_oecd   

                                      Representation  \
0  [also, government, technologies, report, may, ...   
1  [system, also, may, protection, personal, stan...   
2  [uk, government, strategy, innovation, health,...   
3  [ethics, ethical, machine, system, may, also, 

In [169]:
print(topic_model.get_topic(-1)) 

[('also', 0.012075053366697045), ('government', 0.011012281133032223), ('technologies', 0.01053147377932307), ('report', 0.010365084364266171), ('may', 0.010207114452158206), ('social', 0.009950220350407444), ('algorithms', 0.009826026538115519), ('work', 0.009598106524077629), ('services', 0.009059133029213729), ('could', 0.008648757892195335)]


In [170]:
# Visualisation des topics
topic_model.visualize_topics()


In [171]:

# Visualisation des hiérarchies entre les topics
topic_model.visualize_hierarchy()

In [172]:

# Visualisation des relations entre les topics
topic_model.visualize_barchart()

In [179]:
topic_model.visualize_heatmap(n_clusters=8)

In [174]:
topic_model.get_representative_docs(0)

["privacy report january 201 8 2 3 contents report ................................ ................................ ................................ ................................ ................. 4 legal sources terminology ................................ ................................ ................................ ............... 4 intelligen ce protecti ................................ ................................ ................................ ..... 5 ntelligence work ................................ ................................ ................................ ............ 7 machine ................................ ................................ ................................ ................................ ................. 7 results ................................ ................................ ................................ ................................ ............. 10 training better ................................ ...........................

In [175]:
df[df["topic"] == -1]

,doc_id,Name of the document,Institution,URL,Authors,Affiliates,Sector,Country,Date,Keywords,...,Unnamed: 28,Unnamed: 29,text,langue,text_processed,tfidf,categorie Institution,theme,topic,probs
3,13,The Toronto Declaration: Protecting the right ...,Access Now,https://www.torontodeclaration.org/declaration...,"Anna Bacciarelli, Joe Westby, Fanny Hidvegi, E...","Access Now, AI Now Institute at New York Unive...",civil society,UK,2018-05-16,NaN,...,NaN,NaN,\n\n\n\n\nThe Toronto Declaration\n\n\n\n\n\n\...,en,toronto declaration skip main content toronto ...,[0. 0. 0. ... 0. 0. 0.],ONG et initiatives de droits humains,Justice and fairness,-1,0.0
9,21,Report on Artificial Intelligence and Human So...,Advisory Board on Artificial Intelligence and ...,https://www8.cao.go.jp/cstp/tyousakai/ai/summa...,NaN,NaN,NaN,Japan,NaN,NaN,...,NaN,NaN,Report on Artificial Intelligence and...,en,report artificial intelligence human society u...,[0. 0. 0. ... 0. 0. 0.],Gouvernements et organismes publics nationaux,Beneficence,-1,0.0
15,28,AI Now 2019 Report,AI Now Institute,https://ainowinstitute.org/wp-content/uploads/...,NaN,NaN,NaN,USA,NaN,NaN,...,NaN,NaN,\n AI Now 2019 Report | 2 AUTHORS A...,en,ai 2019 report 2 authors contributors kate cra...,[0. 0.00410139 0. ... 0. ...,Instituts de recherche et universités,Privacy,-1,0.0
20,35,A toolkit for centering racial equity througho...,AISP,https://aisp.upenn.edu/wp-content/uploads/2022...,NaN,NaN,NaN,USA,NaN,NaN,...,NaN,NaN,Centering Racial Equity Throughout Data Int...,en,centering racial equity throughout data integr...,[0. 0.00248242 0. ... 0. ...,Associations professionnelles et groupes de ré...,Justice and fairness,-1,0.0
24,39,AI and Human Rights,All Tech is Human,https://alltechishuman.org/ai-human-rights-report,NaN,NaN,NaN,USA,NaN,NaN,...,NaN,NaN,\n\n\n\n\n\n\nAI and Human Rights: Building a ...,en,ai human rights building tech future aligned p...,[0. 0. 0. ... 0. 0. 0.],Associations professionnelles et groupes de ré...,Sustainable,-1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,718,AI procurement in a box: AI government procure...,World Economic Forum (WEF),https://www3.weforum.org/docs/WEF_AI_Procureme...,NaN,NaN,NaN,international,NaN,NaN,...,NaN,NaN,AI Procurement in a Box: Project overviewUnl...,en,ai procurement box project overviewunlocking p...,[0. 0. 0. ... 0. 0. 0.],Associations professionnelles et groupes de ré...,Trust,-1,0.0
449,719,Artificial Intelligence for Children - Toolkit,World Economic Forum (WEF),https://www3.weforum.org/docs/WEF_Artificial_I...,NaN,NaN,NaN,international,NaN,NaN,...,NaN,NaN,AI Procurement in a Box: Project overviewUnl...,en,ai procurement box project overviewunlocking p...,[0. 0. 0. ... 0. 0. 0.],Associations professionnelles et groupes de ré...,Trust,-1,0.0
450,720,Guidelines for AI procurement,Department for Digital,https://www.gov.uk/government/publications/gui...,NaN,NaN,NaN,UK,NaN,NaN,...,NaN,NaN,White Paper Guidelines for AI Procurement Sept...,en,white paper guidelines ai procurement septembe...,[0. 0.00773414 0. ... 0. ...,Gouvernements et organismes publics nationaux,Responsibility,-1,0.0
451,721,How to prevent Discriminatory Outcomes in Mach...,World Economic Forum (WEF),http://www3.weforum.org/docs/WEF_40065_White_P...,NaN,NaN,NaN,international,NaN,NaN,...,NaN,NaN,White Paper Guidelines for AI Procurement Sept...,en,white paper guidelines ai procurement septembe...,[0. 0.00773414 0. ... 0. ...,Associations professionnelles et groupes de ré...,Responsibility,-1,0.0


In [176]:
df.head()


,doc_id,Name of the document,Institution,URL,Authors,Affiliates,Sector,Country,Date,Keywords,...,Unnamed: 28,Unnamed: 29,text,langue,text_processed,tfidf,categorie Institution,theme,topic,probs
0,3,Intel Recommends Public Policy Principles for ...,Intel,https://community.intel.com/t5/Blogs/Intel/Pol...,Naveen Rao,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,\n\n\n\tIntel Recommends Public Policy Princip...,en,intel recommends public policy principles arti...,[0.01722362 0. 0. ... 0. ...,Entreprises technologiques et multinationales,Privacy,2,0.609578
1,9,Artificial intelligence in Healthcare,Academy of Medical Royal Colleges,https://www.aomrc.org.uk/wp-content/uploads/20...,NaN,NaN,NaN,UK,NaN,NaN,...,NaN,NaN,Artificial Intelligence in HealthcareJanuary /...,en,artificial intelligence healthcarejanuary 2019...,[0. 0.00872152 0. ... 0. ...,Instituts de recherche et universités,Trust,1,1.000000
2,12,Human rights in the age of Artificial Intellig...,Access Now,https://www.accessnow.org/cms/assets/uploads/2...,.,.,NaN,NaN,NaN,NaN,...,NaN,NaN,HUMAN RIGHTS IN THE AGE OF ARTIFICIAL INTELLI...,en,human rights age artificial intelligence repor...,[0. 0.0087539 0. ... 0. 0...,ONG et initiatives de droits humains,Justice and fairness,4,1.000000
3,13,The Toronto Declaration: Protecting the right ...,Access Now,https://www.torontodeclaration.org/declaration...,"Anna Bacciarelli, Joe Westby, Fanny Hidvegi, E...","Access Now, AI Now Institute at New York Unive...",civil society,UK,2018-05-16,NaN,...,NaN,NaN,\n\n\n\n\nThe Toronto Declaration\n\n\n\n\n\n\...,en,toronto declaration skip main content toronto ...,[0. 0. 0. ... 0. 0. 0.],ONG et initiatives de droits humains,Justice and fairness,-1,0.000000
4,14,Europe’s approach to artificial intelligence: ...,AccessNow,https://www.accessnow.org/wp-content/uploads/2...,NaN,NaN,NaN,USA,2020-12,NaN,...,NaN,NaN,accessnow.org EUROPE’S APPROACH TO ARTIFICIAL ...,en,accessnow.org europe ’ approach artificial int...,[0. 0.0035214 0. ... 0. 0...,ONG et initiatives de droits humains,Transparency,7,1.000000
